<a href="https://colab.research.google.com/github/rahiakela/data-learning-research-and-practice/blob/main/deep-learning-with-python-by-francois-chollet/11-deep-learning-for-text/01_words_representing_approach_sets_and_sequences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Words representing approache: Sets and sequences

A much more problematic question,
however, is how to encode the way words are woven into sentences: word order.

The problem of order in natural language is an interesting one: unlike the steps of a timeseries, words in a sentence don’t have a natural, canonical order.

The simplest thing you could do is just discard order and
treat text as an unordered set of words—this gives you **bag-of-words models**.

You could also decide that words should be processed strictly in the order in which they appear, one at a time, like steps in a timeseries—you could then leverage the **recurrent models**.

Finally, a hybrid approach is also possible: the **Transformer architecture** is technically order-agnostic, yet it injects word-position information into
the representations it processes, which enables it to simultaneously look at different parts of a sentence, while still being order-aware. 

Because they take into account word order, **both RNNs and Transformers are called sequence models.**

We’ll demonstrate each approach on a well-known text classification benchmark:
the IMDB movie review sentiment-classification dataset.

Let’s process the raw IMDB text data, just like you would do when approaching a new text-classification problem in the real world.





##Setup

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import os, pathlib, shutil, random

Let’s start by downloading the dataset from the Stanford page and uncompressing it:

In [2]:
%%shell

curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
tar -xf aclImdb_v1.tar.gz

# delete unwanted file and subdirectory
rm -rf aclImdb/train/unsup
rm -rf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  16.3M      0  0:00:04  0:00:04 --:--:-- 18.3M


In [3]:
# take a look at the content of a few of these text files
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

##Preparing the IMDB movie reviews data

For instance, the `train/pos/` directory contains a set of `12,500` text files, each of which
contains the text body of a positive-sentiment movie review to be used as training data.
The negative-sentiment reviews live in the “neg” directories. 

In total, there are `25,000`
text files for training and another 25,000 for testing.

Next, let’s prepare a validation set by setting apart 20% of the training text files in a new directory, `aclImdb/val`:

In [4]:
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir/"val"
train_dir = base_dir/"train"

for category in ("neg", "pos"):
  os.makedirs(val_dir/category)
  files = os.listdir(train_dir/category)
  # Shuffle the list of training files using a seed, to ensure we get the same validation set every time we run the code
  random.Random(1337).shuffle(files)
  # Take 20% of the training files to use for validation
  num_val_samples = int(0.2 * len(files))
  val_files = files[-num_val_samples:]
  for fname in val_files:
    # Move the files to aclImdb/val/neg and aclImdb/val/pos
    shutil.move(train_dir/category/fname, val_dir/category/fname)

Remember how, we used the `image_dataset_from_directory` utility to
create a batched Dataset of images and their labels for a directory structure? You can do the exact same thing for text files using the `text_dataset_from_directory` utility.

Let’s create three Dataset objects for training, validation, and testing:

In [5]:
batch_size = 32

train_ds = keras.utils.text_dataset_from_directory("aclImdb/train", batch_size=batch_size)
val_ds = keras.utils.text_dataset_from_directory("aclImdb/val", batch_size=batch_size)
test_ds = keras.utils.text_dataset_from_directory("aclImdb/test", batch_size=batch_size)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


These datasets yield inputs that are TensorFlow `tf.string` tensors and targets that are `int32` tensors encoding the value “0” or “1.”

In [6]:
for inputs, targets in train_ds:
  print("inputs.shape:", inputs.shape)
  print("inputs.dtype:", inputs.dtype)
  print("targets.shape:", targets.shape)
  print("targets.dtype:", targets.dtype)
  print("inputs[0]:", inputs[0])
  print("targets[0]:", targets[0])
  break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b'This rip off of the 1984 hit "Gremlins" is quite possibly the biggest train wreck of a movie ever made. Even for a \'B\' grade movie, all other cheap horror movies on the same platform completely dwarf this movie in terms of plot, acting, and goodness.<br /><br />It begins with a random old security guard and the younger punky security guard whose name is of no importance. Why? Because a few minutes into the film he walks into the \'forbidden\' safe, and is killed whilst living out his fantasy of being a rock star in a cheap pub.<br /><br />This is just an appetizer for the scat-filled main course. The main character, KEVIN, struggles various times to prove himself as more than a total pussy. Perhaps he succeeds within the film, but to the audience he proves himself as nothing more than a bad actor. Kevin gets himself a job with the old security guard, and is 

All set. Now let’s try learning something from this data.

##Processing words as a set: The bag-of-words approach

The simplest way to encode a piece of text for processing by a machine learning
model is to discard order and treat it as a set (a “bag”) of tokens. 

You could either look at individual words (unigrams), or try to recover some local order information by looking at groups of consecutive token (N-grams).

###Single words with binary encoding

If you use a bag of single words, the sentence “the cat sat on the mat” becomes-

```python
{"cat", "mat", "on", "sat", "the"}
```

The main advantage of this encoding is that you can represent an entire text as a single
vector, where each entry is a presence indicator for a given word. 

For instance,
using binary encoding (multi-hot), you’d encode a text as a vector with as many
dimensions as there are words in your vocabulary—with 0s almost everywhere and
some 1s for dimensions that encode words present in the text.

First, let’s process our raw text datasets with a TextVectorization layer so that they yield multi-hot encoded binary word vectors.

In [7]:
# Limit the vocabulary to the 20,000 most frequent words.In general, 20,000 is the right vocabulary size for text classification.
text_vectorization = layers.TextVectorization(max_tokens=20000, output_mode="multi_hot") # Encode the output tokens as multi-hot binary vectors

# Prepare a dataset that only yields raw text inputs (no labels).
text_only_train_ds = train_ds.map(lambda x, y: x)

# Use that dataset to index the dataset vocabulary via the adapt() method
text_vectorization.adapt(text_only_train_ds)

In [8]:
# Prepare processed versions of our training, validation, and test dataset.
binary_1gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
binary_1gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
binary_1gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

You can try to inspect the output of one of these datasets.

In [9]:
for inputs, targets in binary_1gram_train_ds:
  print("inputs.shape:", inputs.shape)
  print("inputs.dtype:", inputs.dtype)
  print("targets.shape:", targets.shape)
  print("targets.dtype:", targets.dtype)
  print("inputs[0]:", inputs[0])
  print("targets[0]:", targets[0])
  break

inputs.shape: (32, 20000)
inputs.dtype: <dtype: 'float32'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor([1. 1. 1. ... 0. 0. 0.], shape=(20000,), dtype=float32)
targets[0]: tf.Tensor(0, shape=(), dtype=int32)


Next, let’s write a reusable model-building function that we’ll use in all of our experiments.

In [10]:
def get_model(max_tokens=20000, hidden_dim=16):
  inputs = keras.Input(shape=(max_tokens, ))
  x = layers.Dense(hidden_dim, activation="relu")(inputs)
  x = layers.Dropout(0.5)(x)
  outputs = layers.Dense(1, activation="sigmoid")(x)
  model = keras.Model(inputs, outputs)
  model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
  return model

Finally, let’s train and test our model.

In [11]:
model = get_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense (Dense)               (None, 16)                320016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________


In [12]:
callbacks = [keras.callbacks.ModelCheckpoint("binary_1gram.keras", save_best_only=True)]

We call `cache()` on the datasets to cache them in memory: this way, we will only do the preprocessing once, during the first epoch, and we’ll reuse the preprocessed texts for the following epochs. This can only be done if the data is small enough to fit in memory.

In [13]:
model.fit(binary_1gram_train_ds.cache(),
          validation_data=binary_1gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)

Epoch 1/10
625/625 [==============================] - 18s 26ms/step - loss: 0.3967 - accuracy: 0.8352 - val_loss: 0.3001 - val_accuracy: 0.8804
Epoch 2/10
625/625 [==============================] - 7s 11ms/step - loss: 0.2655 - accuracy: 0.9009 - val_loss: 0.3007 - val_accuracy: 0.8848
Epoch 3/10
625/625 [==============================] - 7s 11ms/step - loss: 0.2361 - accuracy: 0.9197 - val_loss: 0.3113 - val_accuracy: 0.8894
Epoch 4/10
625/625 [==============================] - 6s 10ms/step - loss: 0.2214 - accuracy: 0.9257 - val_loss: 0.3351 - val_accuracy: 0.8880
Epoch 5/10
625/625 [==============================] - 6s 9ms/step - loss: 0.2177 - accuracy: 0.9299 - val_loss: 0.3378 - val_accuracy: 0.8902
Epoch 6/10
625/625 [==============================] - 5s 8ms/step - loss: 0.2155 - accuracy: 0.9319 - val_loss: 0.3459 - val_accuracy: 0.8912
Epoch 7/10
625/625 [==============================] - 5s 8ms/step - loss: 0.2030 - accuracy: 0.9375 - val_loss: 0.3664 - val_accuracy: 0.8860
E

In [14]:
model = keras.models.load_model("binary_1gram.keras")
print(f"Test acc: {model.evaluate(binary_1gram_test_ds)[1]:.3f}")

782/782 [==============================] - 10s 13ms/step - loss: 0.2949 - accuracy: 0.8854
Test acc: 0.885


This gets us to a test accuracy of 89.2%: not bad! Note that in this case, since the dataset
is a balanced two-class classification dataset (there are as many positive samples as
negative samples), the “naive baseline” we could reach without training an actual model
would only be 50%. 

Meanwhile, the best score that can be achieved on this dataset
without leveraging external data is around 95% test accuracy.

###Bigrams with binary encoding